---
title: "Decision Trees and Random Forests"
jupyter: python3
bibliography: references.bib
nocite: |
  @Hastie2009
---

## Outline

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/14-Classification-I-Decision-Trees.ipynb)

- Build a decision tree manually
- Look at single and collective impurity measures
- Selecting splitting attributes and test conditions
- Scikit-learn implementation
- Model training and evaluation
- Bias and Variance
- Random forests

## Introduction

In [ ]:
#| echo: false
from sklearn import tree

We'll now start looking into how to build models to predict an outcome variable from labeled data.

**Classification** problems:

- predict a category
- e.g., spam/not spam, fraud/not fraud, default/not default, malignant/benign, etc.

**Regression** problems:

- predict a numeric value
- e.g., price of a house, salary of a person, etc.


## Loan Default Example

We'll use an example from [@Tan2018].

![](figs/L14-terrier-savings-logo.png){height="200px"}

You are a loan officer at **Terrier Savings and Loan**. 

You have a dataset on loans that you have made in the past.

You want to build a model to predict whether a loan will default.

## Loans Data Set

In [ ]:
#| echo: false
import pandas as pd

import os
import urllib.request

# Check if the directory exists, if not, create it
if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists('data/loans.csv'):
    url = 'https://raw.githubusercontent.com/tools4ds/DS701-Course-Notes/refs/heads/main/ds701_book/data/loans.csv'
    urllib.request.urlretrieve(url, 'data/loans.csv')

loans = pd.read_csv('data/loans.csv', index_col=0)
loans

## Loans Data Set Summary

Here's the summary info of the data set.

In [ ]:
loans.info()

## Convert to Categorical Data Types

Since some of the fields are categorical, let's convert them to categorical data types.

In [ ]:
#| code-fold: false
loans['Home Owner'] = loans['Home Owner'].astype('category')
loans['Marital Status'] = loans['Marital Status'].astype('category')
loans['Defaulted Borrower'] = loans['Defaulted Borrower'].astype('category')
loans.info()

## Simple Model

Looking at the table, let's just start with the simplest model possible and just
predict that no one will default.

So the output of our model is just to always predict "No".

In [ ]:
#| echo: false
#| fig-align: center
import graphviz

dot_data = """
digraph Tree {
    node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="values = [# No, # Yes] =[7,3]\\ndefaulted = No\\nerror = 30%", fillcolor="#ffffff"] ;
}
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

<!-- TODO: Look at this reference to consider pydotplus for size control -->


We see a 30% error rate since 3 out of 10 loans defaulted.

---

Let's split the data based on the "Home Owner" field. (`values = [# No, # Yes]`).

In [ ]:
#| echo: false
#| fig-align: center
import graphviz

dot_data = """
digraph Tree {
    node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Home Owner?\\n---\\nsamples = 10\\nvalues = [7, 3]\\ndefaulted = No\\nerror = 30%", fillcolor="#ffffff"] ;
    1 [label="samples = 3\\nvalue = [3, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    2 [label="samples = 7\\nvalue = [4, 3]\\ndefaulted = No\\nerror = 43%", fillcolor="#ffffff"] ;
    0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="Yes"] ;
    0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="No"] ;}
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

::: {.fragment}
We see that the left node (`Home Owner == Yes`) has a 0% error rate since all the samples are `Defaulted == No`. We don't split this node since all the samples are of the same class. We call this node a **leaf node** and we'll color it orange.
:::

::: {.fragment}
The right node (`Home Owner == No`) has a 43% error rate since 3 out of 7 loans defaulted. 

Let's split this node into two nodes based on the **Marital Status** field.
:::
---

Let's split on the "Marital Status" field.

We see that the 3 defaulted loans are all for single or divorced people. Since the node is
all one class, we don't split this node and we call it a **leaf node**.

In [ ]:
#| echo: false
#| fig-align: center
import graphviz

dot_data = """
digraph Tree {
    node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Home Owner?\\n---\\nsamples = 10\\nvalues = [7, 3]\\ndefaulted = No\\nerror = 30%", fillcolor="#ffffff"] ;
    1 [label="samples = 3\\nvalue = [3, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    2 [label="Marital Status?\\n---\\nsamples = 7\\nvalue = [4, 3]\\ndefaulted = Yes", fillcolor="#ffffff"] ;
    3 [label="samples = 4\\nvalue = [1, 3]\\ndefaulted = Yes\\nerror = 25%", fillcolor="#ffffff"] ;
    4 [label="samples = 3\\nvalue = [3, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="Yes"] ;
    0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="No"] ;
    2 -> 3 [labeldistance=2.5, labelangle=45, headlabel="Single,\\nDivorced"] ;
    2 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="Married"] ;}
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

--- 

We can list the subsets for the two criteria to calculate the error rate.

In [ ]:
#| fig-cap: 'Table: Home Owner == No and Marital Status == Single or Divorced --> Defaulted == Yes'
#| fig-cap-location: top
loans[(loans['Home Owner'] == "No") & (loans['Marital Status'].isin(['Single', 'Divorced']))]

::: {.fragment}
Error rate for predicting `Defaulted == Yes` is 25%.
:::

---

and...

In [ ]:
#| fig-cap: 'Table: Home Owner == No and Marital Status == Married --> Defaulted == No'
#| fig-cap-location: top
loans[(loans['Home Owner'] == "No") & (loans['Marital Status'] == "Married")]

::: {.fragment}
Error rate for predicting `Defaulted == No` is 0%.
:::

---

:::: {.columns}
::: {.column width="50%"}

* Let's try to split on the "Annual Income" field. 

* We see that the person with income of 70K doesn't default, so we split the node into two nodes based on the "Income" field. 

* We arbitrarily pick a threshold of $75K.
:::
::: {.column width="50%"}

In [ ]:
#| echo: false
#| fig-align: center
import graphviz

dot_data = """
digraph Tree {
    graph [size="8,8"];
    node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Home Owner\\n---\\nsamples = 10\\nvalues = [7, 3]\\ndefaulted = No\\nerror = 30%", fillcolor="#ffffff"] ;
    1 [label="samples = 3\\nvalue = [3, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    2 [label="Marital Status\\n---\\nsamples = 7\\nvalue = [4, 3]\\ndefaulted = Yes", fillcolor="#ffffff"] ;
    3 [label="Income <= 75K\\nsamples = 4\\nvalue = [1, 3]\\ndefaulted = Yes\\nerror = 25%", fillcolor="#ffffff"] ;
    4 [label="samples = 3\\nvalue = [3, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    5 [label="samples = 1\\nvalue = [1, 0]\\ndefaulted = No\\nerror = 0%", fillcolor="#e58139"] ;
    6 [label="samples = 3\\nvalue = [0, 3]\\ndefaulted = Yes\\nerror = 0%", fillcolor="#e58139"] ;
    0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="Yes"] ;
    0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="No"] ;
    2 -> 3 [labeldistance=2.5, labelangle=45, headlabel="Single,\\nDivorced"] ;
    2 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="Married"] ;
    3 -> 5 [labeldistance=2.5, labelangle=45, headlabel="Yes"] ;
    3 -> 6 [labeldistance=2.5, labelangle=-45, headlabel="No"] ;}
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)
graph

:::
::::

## Evaluating the Model

::: {.incremental}
* We've dispositioned every data point by walking down the tree to a leaf node.

* How do we know if this tree is good? 

    * We arbitrarily picked the order of the fields to split on.

* Is there a way to systematically pick the order of the fields to split on? 

    * This is called the **splitting criterion**.

* There's also the question of when to stop splitting, or the **stopping criterion**. 

* So far, we stopped splitting when we reached a node of pure class but there are 
reasons to stop splitting even without pure classes, which we'll see later.
:::

## Specifying the Test Condition

Before we continue, we should take a moment to consider how we specify a test condition of a node.

How we specify a test condition depends on the attribute type which can be:

* Binary (Boolean)
* Nominal (Categorical, e.g., cat, dog, bird)
* Ordinal (e.g., Small, Medium, Large)
* Continuous (e.g., 1.5, 2.1, 3.7)

And depends on the number of ways to split:

* __multi-way__
* __binary__

---

For a __Nominal (Categorical)__ attribute:

* In a __Multi-way split__ we can use as many partitions as there are distinct values of the attribute:

In [ ]:
#| echo: false
#| fig-align: center
import graphviz

dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Marital Status", fillcolor="#ffffff"] ;
    1 [label="Single", fillcolor="#ffffff", shape="none"] ;
    2 [label="Divorced", fillcolor="#ffffff", shape="none"] ;
    3 [label="Married", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ;
    0 -> 3 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

---

For a __Nominal (Categorical)__ attribute:

* In a __Binary split__ we divide the values into two groups.  

* In this case, we need to find an optimal partitioning of values into groups, which we discuss shortly.

::: {layout-ncol=3}

In [ ]:
#| echo: false
import graphviz

dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Marital Status", fillcolor="#ffffff"] ;
    1 [label="Single,\\nDivorced", fillcolor="#ffffff", shape="none"] ;
    2 [label="Married", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Marital Status", fillcolor="#ffffff"] ;
    1 [label="Single", fillcolor="#ffffff", shape="none"] ;
    2 [label="Married,\\nDivorced", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Marital Status", fillcolor="#ffffff"] ;
    1 [label="Single,\\nMarried", fillcolor="#ffffff", shape="none"] ;
    2 [label="Divorced", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

:::

---

For an __Ordinal__ attribute, we can use a multi-way split with as many partitions
as there are distinct values.

In [ ]:
#| echo: false
#| fig-align: center
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Shirt\\nSize", fillcolor="#ffffff"] ;
    1 [label="Small", fillcolor="#ffffff", shape="none"] ;
    2 [label="Medium", fillcolor="#ffffff", shape="none"] ;
    3 [label="Large", fillcolor="#ffffff", shape="none"] ;
    4 [label="X-Large", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ;
    0 -> 3 ;
    0 -> 4 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

---

Or we can use a binary split as long we preserve the ordering of the values.

::: {layout-ncol=2}

In [ ]:
#| echo: false
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Shirt\\nSize", fillcolor="#ffffff"] ;
    1 [label="Small,\\nMedium", fillcolor="#ffffff", shape="none"] ;
    2 [label="Large,\\nX-Large", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Shirt\\nSize", fillcolor="#ffffff"] ;
    1 [label="Small", fillcolor="#ffffff", shape="none"] ;
    2 [label="Medium, Large,\\nX-Large", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

:::

::: {.callout-warning}
Be careful not to violate the ordering of values such as {Small, Large} and {Medium, X-Large}.
:::

---

A __Continuous__ attribute can be handled two ways:

::: {layout-ncol=2}

In [ ]:
#| echo: false
#| fig-cap: 'It can be thresholded to form a binary split.'
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Income\\n<= 75K", fillcolor="#ffffff"] ;
    1 [label="Yes", fillcolor="#ffffff", shape="none"] ;
    2 [label="No", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
#| fig-cap: 'Or it can be split into contiguous ranges to form an ordinal categorical attribute.'
dot_data = """
digraph Tree {
    node [shape=oval, style="filled, rounded", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Income", fillcolor="#ffffff"] ;
    1 [label="< 10K", fillcolor="#ffffff"shape="none"] ;
    2 [label="[10K, 25K)", fillcolor="#ffffff", shape="none"] ;
    3 [label="[25K, 50K)", fillcolor="#ffffff", shape="none"] ;
    4 [label="[50K, 75K)", fillcolor="#ffffff", shape="none"] ;
    5 [label="> 75K", fillcolor="#ffffff", shape="none"] ;
    0 -> 1 ;
    0 -> 2 ;
    0 -> 3 ;
    0 -> 4 ;
    0 -> 5 ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

:::

::: {.fragment}

Note that finding good partitions for $k$ nominal attributes can be expensive, $\mathcal{O}(2^k)$, 
possibly involving combinatorial searching of groupings.  

However for ordinal or continuous attributes, sweeping through a range of $n$
threshold values can be more efficient if $n \approx k$. $\mathcal{O}(n)$ for a sorted list.
:::

## Selecting Attribute and Test Condition

::: {.incremental}
* Ideally, we want to pick attributes and test conditions that maximize the homogeneity of the splits.

* We can use an **impurity index** to measure the homogeneity in a node.

* We'll look at ways of measuring impurity of a node and then collective impurity of its child nodes.
:::

## Impurity Measures

The following are three impurity indices:

$$
\begin{aligned}
\textnormal{Gini index} &= 1 - \sum_{i=0}^{c-1}  p_i(t)^2 \\
\textnormal{Entropy} &= -\sum_{i=0}^{c-1}  p_i(t) \log_2 p_i(t) \\
\textnormal{Classification error} &= 1 - \max_i p_i(t)
\end{aligned}
$$

where $p_i(t)$ is the **relative frequency** of training instances of class $i$ at a node $t$ and $c$ is the number of classes.

::: {.callout-note}
By convention, we set $0 \log_2 0 = 0$ in entropy calculations.
:::


## Impurity Measures

The following are three impurity indices:

$$
\begin{aligned}
\textnormal{Gini index} &= 1 - \sum_{i=0}^{c-1}  p_i(t)^2 \\
\textnormal{Entropy} &= -\sum_{i=0}^{c-1}  p_i(t) \log_2 p_i(t) \\
\textnormal{Classification error} &= 1 - \max_i p_i(t)
\end{aligned}
$$


All three impurity indices equal 0 when all the records at a node belong to the same class.

All three impurity indices reach their maximum value when the classes are evenly distributed among the child nodes.

---

We can plot the three impurity indices to get a sense of how they behave for **binary classification** problems.

In [ ]:
#| echo: false
#| fig-align: center
import numpy as np
import matplotlib.pyplot as plt

# Define probability values
p = np.linspace(0, 1, 100)

# Calculate impurity measures
entropy = -p * np.log2(p + 1e-9) - (1 - p) * np.log2(1 - p + 1e-9)  # Adding small epsilon to avoid log(0)
gini = 2 * p * (1 - p)
misclassification_error = 1 - np.maximum(p, 1 - p)

# Create the plot
plt.figure(figsize=(8, 5))
plt.plot(p, entropy, label='Entropy', linewidth=2)
plt.plot(p, gini, '--', label='Gini', linewidth=2)
plt.plot(p, misclassification_error, '-.', label='Misclassification error', linewidth=2)

# Add labels and title
plt.xlabel('p')
plt.ylabel('Impurity')
plt.title('Comparison among the impurity measures for binary classification problems')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

They all maintain the same ordering for every relative frequency, i.e., Entropy > Gini > Misclassification error.


## Impurity Example 1



| Node $N_1$ | Count | p |
| --- | --- | --- |
| Class=0 | 0 | $0/6 = 0$ |
| Class=1 | 6 | $6/6 = 1$ |

<br>

$$
\begin{aligned}
\textnormal{Gini} &= 1 - \left(\frac{0}{6}\right)^2 - \left(\frac{6}{6}\right)^2 = 0 \\
\textnormal{Entropy} &= -\left(\frac{0}{6} \log_2 \frac{0}{6} + \frac{6}{6} \log_2 \frac{6}{6}\right) = 0 \\
\textnormal{Error} &= 1 - \max\left[\frac{0}{6}, \frac{6}{6}\right] = 0
\end{aligned}
$$



## Impurity Example 2


| Node $N_2$ | Count | p |
| --- | --- | --- |
| Class=0 | 1 | $1/6 = 0.167$ |
| Class=1 | 5 | $5/6 = 0.833$ |

<br>

$$
\begin{aligned}
\textnormal{Gini} &= 1 - \left(\frac{1}{6}\right)^2 - \left(\frac{5}{6}\right)^2 = 0.278 \\
\textnormal{Entropy} &= -\left(\frac{1}{6} \log_2 \frac{1}{6} + \frac{5}{6} \log_2 \frac{5}{6}\right) = 0.650 \\
\textnormal{Error} &= 1 - \max\left[\frac{1}{6}, \frac{5}{6}\right] = 0.167
\end{aligned}
$$



## Impurity Example 3


| Node $N_3$ | Count |
| --- | --- |
| Class=0 | 3 |
| Class=1 | 3 |

<br>

$$
\begin{aligned}
\textnormal{Gini} &= 1 - \left(\frac{3}{6}\right)^2 - \left(\frac{3}{6}\right)^2 = 0.5 \\
\textnormal{Entropy} &= -\left(\frac{3}{6} \log_2 \frac{3}{6} + \frac{3}{6} \log_2 \frac{3}{6}\right) = 1 \\
\textnormal{Error} &= 1 - \max\left[\frac{3}{6}, \frac{3}{6}\right] = 0.5
\end{aligned}
$$

## Impurity Class Exercise

Calculate Gini impurity for the following node:

<br>

| Node $N_4$ | Count | p |
| --- | --- | --- |
| Class=0 | 3 |  |
| Class=1 | 6 |  |
| Class=2 | 1 |  |

<br>

$$
\textnormal{Gini} = 
$$

<!--
$$
\begin{aligned}
\textnormal{Gini} &= 1 - \left(\frac{3}{10}\right)^2 - \left(\frac{6}{10}\right)^2 - \left(\frac{1}{10}\right)^2 \\
&= 1 - \left(\frac{9}{100}\right) - \left(\frac{36}{10}\right) - \left(\frac{1}{100}\right) \\
&= 1 - 0.09 - 0.36 - 0.01 \\
&= 0.54
\end{aligned}
$$
-->


## Collective Impurity of Child Nodes

We can compute the collective impurity of child nodes by taking a weighted sum of the impurities of the child nodes.

$$
I(\textnormal{children}) = \sum_{j=1}^{k} \frac{N(v_j)}{N}\; I(v_j)
$$

Here we split $N$ training instances into $k$ child nodes, $v_j$ for $j=1, \ldots, k$.

$N(v_j)$ is the number of training instances at child node $v_j$ and $I(v_j)$ is the impurity at child node $v_j$.

## Impurity Example

Let's compute collective impurity on our loans dataset to see which feature to split on.

::: {layout-ncol=3}

In [ ]:
#| echo: false
#| fig-cap: '(a) Collective Entropy: 0.690'
dot_data = """
digraph Tree {
    node [shape=oval, style="filled", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Home\\nOwner", fillcolor="#ffffff"] ;
    1 [label="Yes: 0\\nNo: 3", fillcolor="#ffffff", shape="square"] ;
    2 [label="Yes: 3\\nNo: 4", fillcolor="#ffffff", shape="square"] ;
    0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="Yes"] ;
    0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="No"] ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
#| fig-cap: '(b) Collective Entropy: 0.686'
dot_data = """
digraph Tree {
    node [shape=oval, style="filled", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="Marital\\nStatus", fillcolor="#ffffff"] ;
    1 [label="Yes: 2\\nNo: 3", fillcolor="#ffffff", shape="square"] ;
    2 [label="Yes: 0\\nNo: 3", fillcolor="#ffffff", shape="square"] ;
    3 [label="Yes: 1\\nNo: 1", fillcolor="#ffffff", shape="square"] ;
    0 -> 1 [xlabel="Single"] ;
    0 -> 2 [label="Married"] ;
    0 -> 3 [xlabel="Divorced"] ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

In [ ]:
#| echo: false
#| fig-cap: '(c) Collective Entropy index: 0.00'
dot_data = """
digraph Tree {
    node [shape=oval, style="filled", color="black", fontname="helvetica"] ;
    edge [fontname="helvetica"] ;
    0 [label="ID", fillcolor="#ffffff"] ;
    1 [label="Yes: 0\\nNo: 1", fillcolor="#ffffff", shape="square"] ;
    2 [label="...", fillcolor="#ffffff", shape="none"] ;
    3 [label="Yes: 1\\nNo: 0", fillcolor="#ffffff", shape="square"] ;
    4 [label="Yes: 0\\nNo: 1", fillcolor="#ffffff", shape="square"] ;
    5 [label="Yes: 1\\nNo: 0", fillcolor="#ffffff", shape="square"] ;
    0 -> 1 [xlabel="1"] ;
    0 -> 2 [color="white"] ;
    0 -> 3 [xlabel="8"] ;
    0 -> 4 [xlabel="9"] ;
    0 -> 5 [xlabel="10"] ; }
"""

# Use graphviz to render the dot file
graph = graphviz.Source(dot_data)  
graph

:::

::: {.callout-tip}
Try calculating the collective Entropy for (a) and (b) and see if you get the same values.
:::

::: {.callout-important}
The collective entropy for (c) is 0. Why would we not want to use this node?
:::

---

There are two ways to overcome this problem. 

1. One way is to _generate only binary decision trees_, thus avoiding the difficulty of handling attributes with varying
   number of partitions. This strategy is employed by decision tree classifiers such as **CART**. 
2. Another way is to modify the splitting criterion to take into account the number of partitions produced by the
   attribute. For example, in the **C4.5** decision tree algorithm, a measure known as **gain ratio** is used to compensate
   for attributes that produce a large number of child nodes.

::: aside
CART stands for Classification And Regression Tree.
:::

## Gain Ratio

See [@Tan2018, Chap. 3, p. 127]:

* Having a low impurity value alone is insufficient to find a good attribute test condition for a node. 
* Having more child nodes can make a decision tree more complex and consequently more susceptible to overfitting. 

Hence, the number of children produced by the splitting attribute should also be taken into consideration while deciding the best attribute test condition. 

## Gain Ratio Formula

$$
\text{Gain ratio} = \frac{\Delta_{\text{info}}}{\text{Split Info}} = \frac{\text{Entropy(Parent)} - \sum_{i=1}^{k} \frac{N(v_i)}{N} \text{Entropy}(v_i)}{- \sum_{i=1}^{k} \frac{N(v_i)}{N} \log_2 \frac{N(v_i)}{N}}
$$

where $N(v_i)$ is the number of instances assigned to node $v_i$ and $k$ is the total number of splits. 

The split information measures the entropy of splitting a node into its child nodes and evaluates if the split results
in a larger number of equally-sized child nodes or not.

## Gain Ratio Example

Let's compare the Gain Ratio for **Home Owner** and **Marital Status** attributes using the loans dataset.

Recall from the earlier example that the parent node has 3 Yes and 7 No defaulters (10 total instances).

**Parent node entropy:**

$$
\text{Entropy(Parent)} = -\frac{3}{10} \log_2 \frac{3}{10} - \frac{7}{10} \log_2 \frac{7}{10} = 0.881
$$

## Ex: Gain Ratio for Home Owner Attribute

The Home Owner attribute splits the data into 2 child nodes:

- Yes: 0 Yes, 3 No (3 instances)
- No: 3 Yes, 4 No (7 instances)

From the earlier calculation, the Information Gain is:

$$
\Delta_{\text{info}} = 0.881 - \left(\frac{3}{10} \times 0 + \frac{7}{10} \times 0.985\right) = 0.192
$$

Now we calculate the Split Information:

$$
\text{Split Info} = -\frac{3}{10} \log_2 \frac{3}{10} - \frac{7}{10} \log_2 \frac{7}{10} = 0.881
$$

Therefore, the Gain Ratio is:

$$
\text{Gain Ratio(Home Owner)} = \frac{0.192}{0.881} = 0.218
$$

## Ex: Gain Ratio for Marital Status Attribute

The Marital Status attribute splits the data into 3 child nodes:

- Single: 2 Yes, 3 No (5 instances)
- Married: 0 Yes, 3 No (3 instances)
- Divorced: 1 Yes, 1 No (2 instances)

From the earlier calculation, the Information Gain is:

$$
\Delta_{\text{info}} = 0.881 - \left(\frac{5}{10} \times 0.971 + \frac{3}{10} \times 0 + \frac{2}{10} \times 1\right) = 0.195
$$

Now we calculate the Split Information:

$$
\text{Split Info} = -\frac{5}{10} \log_2 \frac{5}{10} - \frac{3}{10} \log_2 \frac{3}{10} - \frac{2}{10} \log_2 \frac{2}{10} = 1.486
$$

Therefore, the Gain Ratio is:

$$
\text{Gain Ratio(Marital Status)} = \frac{0.195}{1.486} = 0.131
$$

## Example: Comparison of Gain Ratios

| Attribute | Information Gain | Split Info | Gain Ratio |
|-----------|-----------------|------------|------------|
| Home Owner | 0.192 | 0.881 | **0.218** |
| Marital Status | 0.195 | 1.486 | 0.131 |

Although Marital Status has a slightly higher Information Gain, it has a **lower Gain Ratio** because it splits the data into 3 child nodes rather than 2. 

The Gain Ratio penalizes attributes that create more splits, making **Home Owner** the preferred choice.

## Identifying the Best Attribute Test Condition

![](figs/L14-splitting-criteria-gini.png){width="70%" fig-align="center"}

Here's an example of how to identify the best attribute test condition using the Collective Impurity of
the Gini Impurity index.

In [ ]:
def gini_impurity(yes_count, no_count):
    """Calculate Gini impurity for a node"""
    total = yes_count + no_count
    if total == 0:
        return 0
    
    p_yes = yes_count / total
    p_no = no_count / total
    
    gini = 1 - (p_yes**2 + p_no**2)
    return gini

def weighted_gini(n1_yes, n1_no, n2_yes, n2_no):
    """Calculate weighted average Gini impurity after split"""
    n1_total = n1_yes + n1_no
    n2_total = n2_yes + n2_no
    total = n1_total + n2_total
    
    gini_n1 = gini_impurity(n1_yes, n1_no)
    gini_n2 = gini_impurity(n2_yes, n2_no)
    
    weighted_gini = (n1_total/total) * gini_n1 + (n2_total/total) * gini_n2
    return weighted_gini

# Parent node
parent_yes = 3
parent_no = 7
parent_gini = gini_impurity(parent_yes, parent_no)

print("=" * 60)
print("PARENT NODE")
print("=" * 60)
print(f"Yes: {parent_yes}, No: {parent_no}")
print(f"Gini Impurity: {parent_gini:.3f}")
print()

# Define all four splits
splits = {
    "Home Owner": {
        "N1": {"yes": 0, "no": 3},  # Yes (owns home)
        "N2": {"yes": 3, "no": 4}   # No (doesn't own home)
    },
    "Marital Status (Split 1: Single vs Married+Divorced)": {
        "N1": {"yes": 2, "no": 3},  # Single
        "N2": {"yes": 1, "no": 4}   # Married, Divorced
    },
    "Marital Status (Split 2: Single+Married vs Divorced)": {
        "N1": {"yes": 2, "no": 6},  # Single, Married
        "N2": {"yes": 1, "no": 1}   # Divorced
    },
    "Marital Status (Split 3: Single+Divorced vs Married)": {
        "N1": {"yes": 3, "no": 4},  # Single, Divorced
        "N2": {"yes": 0, "no": 3}   # Married
    }
}

# Calculate for each split
results = []
for split_name, nodes in splits.items():
    n1_yes = nodes["N1"]["yes"]
    n1_no = nodes["N1"]["no"]
    n2_yes = nodes["N2"]["yes"]
    n2_no = nodes["N2"]["no"]
    
    gini_n1 = gini_impurity(n1_yes, n1_no)
    gini_n2 = gini_impurity(n2_yes, n2_no)
    weighted_gini_value = weighted_gini(n1_yes, n1_no, n2_yes, n2_no)
    gini_gain = parent_gini - weighted_gini_value
    
    results.append({
        'name': split_name,
        'weighted_gini': weighted_gini_value,
        'gini_gain': gini_gain,
        'gini_n1': gini_n1,
        'gini_n2': gini_n2,
        'n1_yes': n1_yes,
        'n1_no': n1_no,
        'n2_yes': n2_yes,
        'n2_no': n2_no
    })
    
    print("=" * 60)
    print(f"SPLIT: {split_name}")
    print("=" * 60)
    print(f"Node N1: Yes={n1_yes}, No={n1_no}, Total={n1_yes+n1_no}")
    print(f"  Gini(N1) = {gini_n1:.3f}")
    print(f"Node N2: Yes={n2_yes}, No={n2_no}, Total={n2_yes+n2_no}")
    print(f"  Gini(N2) = {gini_n2:.3f}")
    print(f"\nWeighted Gini (after split): {weighted_gini_value:.3f}")
    print(f"Gini Gain: {gini_gain:.3f}")
    print()

# Summary: Find best split
print("=" * 60)
print("SUMMARY - BEST SPLIT")
print("=" * 60)
results_sorted = sorted(results, key=lambda x: x['gini_gain'], reverse=True)
for i, result in enumerate(results_sorted, 1):
    print(f"{i}. {result['name']}")
    print(f"   Weighted Gini: {result['weighted_gini']:.3f}, Gini Gain: {result['gini_gain']:.3f}")
print()
print(f"Best split: {results_sorted[0]['name']}")
print(f"Maximum Gini Gain: {results_sorted[0]['gini_gain']:.3f}")

## Splitting Continuous Attributes

For quantitative attributes like _Annual Income_, we need to find some threshold $\tau$ that
minimizes the impurity index.

The following table illustrates the process.

![](figs/L14-splitting-continuous-attribs.png)


**Procedure:**

1. Sort all the training instances by _Annual Income_ in increasing order.
2. Pick thresholds half way between consecutive values.
3. Compute the Gini impurity index for each threshold.
4. Select the threshold that minimizes the Gini impurity index.

In [ ]:
def gini_impurity(yes_count, no_count):
    """Calculate Gini impurity for a node"""
    total = yes_count + no_count
    if total == 0:
        return 0
    
    p_yes = yes_count / total
    p_no = no_count / total
    
    gini = 1 - (p_yes**2 + p_no**2)
    return gini

def weighted_gini_split(left_yes, left_no, right_yes, right_no):
    """Calculate weighted Gini impurity for a binary split"""
    left_total = left_yes + left_no
    right_total = right_yes + right_no
    total = left_total + right_total
    
    if total == 0:
        return 0
    
    gini_left = gini_impurity(left_yes, left_no)
    gini_right = gini_impurity(right_yes, right_no)
    
    weighted_gini = (left_total/total) * gini_left + (right_total/total) * gini_right
    return weighted_gini

# Data from the table
sorted_values = [60, 70, 75, 85, 90, 95, 100, 120, 125, 220]
classes = ['No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No']

# Calculate split positions (midpoints between consecutive values)
split_positions = []
for i in range(len(sorted_values) - 1):
    split_pos = (sorted_values[i] + sorted_values[i+1]) / 2
    split_positions.append(split_pos)

# Parent node statistics
parent_yes = classes.count('Yes')
parent_no = classes.count('No')
parent_gini = gini_impurity(parent_yes, parent_no)

print("=" * 80)
print("CONTINUOUS ATTRIBUTE: Annual Income")
print("=" * 80)
print(f"Parent Node: Yes={parent_yes}, No={parent_no}")
print(f"Parent Gini Impurity: {parent_gini:.3f}")
print()

print("=" * 80)
print("EVALUATING ALL POSSIBLE SPLITS")
print("=" * 80)

best_split = None
best_gini = float('inf')
best_gain = -float('inf')

results = []

for i, split_pos in enumerate(split_positions):
    # Count classes on left (<= split) and right (> split)
    left_yes = 0
    left_no = 0
    right_yes = 0
    right_no = 0
    
    for j, value in enumerate(sorted_values):
        if value <= split_pos:
            if classes[j] == 'Yes':
                left_yes += 1
            else:
                left_no += 1
        else:
            if classes[j] == 'Yes':
                right_yes += 1
            else:
                right_no += 1
    
    # Calculate weighted Gini
    weighted_gini = weighted_gini_split(left_yes, left_no, right_yes, right_no)
    gini_gain = parent_gini - weighted_gini
    
    results.append({
        'split_position': split_pos,
        'left_yes': left_yes,
        'left_no': left_no,
        'right_yes': right_yes,
        'right_no': right_no,
        'weighted_gini': weighted_gini,
        'gini_gain': gini_gain
    })
    
    print(f"Split at {split_pos:6.1f}:")
    print(f"  Left  (<=): Yes={left_yes}, No={left_no}, Total={left_yes+left_no}")
    print(f"  Right ( >): Yes={right_yes}, No={right_no}, Total={right_yes+right_no}")
    print(f"  Weighted Gini: {weighted_gini:.3f}")
    print(f"  Gini Gain: {gini_gain:.3f}")
    
    if weighted_gini < best_gini:
        best_gini = weighted_gini
        best_split = split_pos
        best_gain = gini_gain
    
    print()

print("=" * 80)
print("BEST SPLIT")
print("=" * 80)
print(f"Best Split Position: {best_split}")
print(f"Minimum Weighted Gini: {best_gini:.3f}")
print(f"Maximum Gini Gain: {best_gain:.3f}")

# Show the split details
best_result = [r for r in results if r['split_position'] == best_split][0]
print(f"\nSplit Details (Annual Income <= {best_split}):")
print(f"  Left  (<=): Yes={best_result['left_yes']}, No={best_result['left_no']}")
print(f"  Right ( >): Yes={best_result['right_yes']}, No={best_result['right_no']}")

## Run Decision Tree on Loans Data Set

Let's run the Scikit-learn Decision Tree, `sklearn.tree`, on the loans data set.

`sklearn.tree` requires all fields to be numeric.

So first we have to convert the categorical fields to category index numeric fields.

In [ ]:
#| code-fold: false
loans['Defaulted Borrower'] = loans['Defaulted Borrower'].cat.codes
loans['Home Owner'] = loans['Home Owner'].cat.codes
loans['Marital Status'] = loans['Marital Status'].cat.codes
loans.head()

---

Then the independent variables are all the fields except the "Defaulted Borrower" field, which we'll assign to `X`.

The dependent variable is the "Defaulted Borrower" field, which we'll assign to `y`.

In [ ]:
#| code-fold: false
from sklearn import tree

X = loans.drop('Defaulted Borrower', axis=1)
y = loans['Defaulted Borrower']

<br>

:::: {.columns}
::: {.column width="50%"}
`X`:

In [ ]:
#| echo: false
X.info()

:::
::: {.column width="50%"}
`y`:

In [ ]:
#| echo: false
y.info()

:::
::::


---

Let's fit a decision tree to the data.

In [ ]:
#| code-fold: false
clf = tree.DecisionTreeClassifier(criterion='gini', random_state=42)
clf = clf.fit(X, y)

Let's plot the tree.

In [ ]:
#| echo: false
annotations = tree.plot_tree(clf, 
               filled=True, 
               rounded=True,
               # max_depth=2,
               # impurity=False,
               feature_names=loans.drop('Defaulted Borrower', axis=1).columns,
               class_names=['No', 'Yes'])

Interestingly, the tree was built using only the Income field.

That's arguably an advantage of Decision Trees: they automatically perform feature selection.


## Ensemble Methods

(See @Tan2018, Chapter 4)

Motivated around the idea that combining several noisy classifiers can result in a better prediction
under certain conditions.

* The base classifiers are independent
* The base classifiers are noisy (high variance)
* The base classifiers are low (ideally zero) bias

## Bias and Variance

__Bias__

* Definition: Error due to overly simplistic models.
* High bias: Model underfits the data.
    * Example: Shallow decision trees.
* Low bias: Model accurately captures the underlying patterns in the data.
    * Example: Deep decision trees.

__Variance__

* Definition: Error due to overly complex models.
* High Variance: Model overfits the data.
    * Example: Deep decision trees.
* Low variance: Model predictions are stable and consistent across different training datasets.

---

![](figs/bias_variance_tradeoff.png){fig-align="center" width="60%"}

## Bias Variance Trade-Off

Goal: Find a balance to minimize total error.

Bias-Variance Trade-off: Low bias and low variance are ideal but challenging to achieve simultaneously.

![[Source](https://serokell.io/blog/bias-variance-tradeoff)](figs/bias_variance_tradeoff2.png){fig-align="center" width="60%"}

## Random Forests

Random forests are an ensemble of decision trees that:

* Construct a set of base classifiers from random sub-samples of the training data.
* Train each base classifier to completion.
* Take a majority vote of the base classifiers to form the final prediction.

## Titanic Example

We'll use the [Titanic data set](https://www.kaggle.com/competitions/titanic) and
excerpts of this [Kaggle tutorial](https://www.kaggle.com/code/jhoward/how-random-forests-really-work/)
to illustrate the concepts of overfitting and random forests.

In [ ]:
import pandas as pd

import os
import urllib.request

# Check if the directory exists, if not, create it
if not os.path.exists('data/titanic'):
    os.makedirs('data/titanic')

if not os.path.exists('data/titanic/train.csv'):
    url = 'https://raw.githubusercontent.com/tools4ds/DS701-Course-Notes/refs/heads/main/ds701_book/data/titanic/train.csv'
    urllib.request.urlretrieve(url, 'data/titanic/train.csv')

df_train = pd.read_csv('data/titanic/train.csv', index_col='PassengerId')

if not os.path.exists('data/titanic/test.csv'):
    url = 'https://raw.githubusercontent.com/tools4ds/DS701-Course-Notes/refs/heads/main/ds701_book/data/titanic/test.csv'
    urllib.request.urlretrieve(url, 'data/titanic/test.csv')

df_test = pd.read_csv('data/titanic/test.csv', index_col='PassengerId')

---

Let's look at the training data.

In [ ]:
df_train.info()

We can see that there are 891 entries with 11 fields. 'Age', 'Cabin', and 'Embarked' have missing values.

---

In [ ]:
df_train.head()

---

Let's look at the test data.

In [ ]:
df_test.info()
df_test.head()

There are 418 entries in the test set with same fields except for 'Survived', which is what we need to predict.

---

We'll do some data cleaning and preparation.

In [ ]:
import numpy as np

def proc_data(df):
    df['Fare'] = df.Fare.fillna(0)
    df.fillna(modes, inplace=True) # Fill missing values with the mode
    df['LogFare'] = np.log1p(df['Fare'])  # Create a new column for the log of the fare + 1
    df['Embarked'] = pd.Categorical(df.Embarked)  # Convert to categorical
    df['Sex'] = pd.Categorical(df.Sex)  # Convert to categorical

modes = df_train.mode().iloc[0] # Get the mode for each column

proc_data(df_train)
proc_data(df_test)

Look at the dataframes again.

:::: {.columns}
::: {.column width="50%"}

In [ ]:
df_train.info()

:::
::: {.column width="50%"}

In [ ]:
df_test.info()

:::
::::

---

We'll create lists of features by type.

In [ ]:
#| code-fold: false
cats=["Sex","Embarked"]  # Categorical
conts=['Age', 'SibSp', 'Parch', 'LogFare',"Pclass"]  # Continuous
dep="Survived"  # Dependent variable

---

Let's explore some fields starting with survival rate by gender.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig,axs = plt.subplots(1,2, figsize=(11,5))
sns.barplot(data=df_train, y='Survived', x="Sex", ax=axs[0], hue="Sex", palette=["#3374a1","#e1812d"]).set(title="Survival rate")
sns.countplot(data=df_train, x="Sex", ax=axs[1], hue="Sex", palette=["#3374a1","#e1812d"]).set(title="Histogram");

Indeed, "women and children first" was enforced on the Titanic.

---

Since we don't have labels for the test data, we'll split the training data into training and validation.

In [ ]:
#| code-fold: false
from numpy import random
from sklearn.model_selection import train_test_split

random.seed(42)
trn_df,val_df = train_test_split(df_train, test_size=0.25)

# Replace categorical fields with numeric codes
trn_df[cats] = trn_df[cats].apply(lambda x: x.cat.codes)
val_df[cats] = val_df[cats].apply(lambda x: x.cat.codes)

---

Let's split the independent (input) variables from the dependent (output) variable.

In [ ]:
#| code-fold: false
def xs_y(df):
    xs = df[cats+conts].copy()
    return xs,df[dep] if dep in df else None

trn_xs,trn_y = xs_y(trn_df)
val_xs,val_y = xs_y(val_df)

---

Here's the predictions for our extremely simple model, where `female` is coded as `0`:

In [ ]:
#| code-fold: false
preds = val_xs.Sex==0

We'll use mean absolute error to measure how good this model is:

In [ ]:
#| code-fold: false
from sklearn.metrics import mean_absolute_error
mean_absolute_error(val_y, preds)

---

Alternatively, we could try splitting on a continuous column. We have to use a somewhat different chart to see how this might work -- here's an example of how we could look at `LogFare`:

In [ ]:
df_fare = trn_df[trn_df.LogFare>0]
fig,axs = plt.subplots(1,2, figsize=(11,5))
sns.boxenplot(data=df_fare, x=dep, y="LogFare", ax=axs[0], hue=dep, palette=["#3374a1","#e1812d"])
sns.kdeplot(data=df_fare, x="LogFare", ax=axs[1]);

The [boxenplot](https://seaborn.pydata.org/generated/seaborn.boxenplot.html) above shows quantiles of `LogFare` for each group of `Survived==0` and `Survived==1`. 

It shows that the average `LogFare` for passengers that didn't survive is around `2.5`, and for those that did it's around `3.2`. 

So it seems that people that paid more for their tickets were more likely to get put on a lifeboat.

---

Let's create a simple model based on this observation:

In [ ]:
#| code-fold: false
preds = val_xs.LogFare>2.7

...and test it out:

In [ ]:
#| code-fold: false
mean_absolute_error(val_y, preds)

This is quite a bit less accurate than our model that used `Sex` as the single binary split.

## Full Decision Tree

Ok. Let's build a decision tree model using all the features.

In [ ]:
#| code-fold: false
clf = tree.DecisionTreeClassifier(criterion='gini', random_state=42)
clf = clf.fit(trn_xs, trn_y)

Let's draw the tree.

In [ ]:
#| code-fold: false
annotations = tree.plot_tree(clf, 
               filled=True, 
               rounded=True,
               feature_names=trn_xs.columns,
               class_names=['No', 'Yes'])

## Full Tree -- Evaluation Error

Let's see how it does on the validation set.

In [ ]:
#| code-fold: false
preds = clf.predict(val_xs)
mean_absolute_error(val_y, preds)

:::: {.fragment}
That is quite a bit worse than splitting on `Sex` alone!!
::::

## Stopping Criteria -- Minimum Samples Split

Let's train the decision tree again but with stopping criteria based on the number of samples in a node.

In [ ]:
#| code-fold: false
clf = tree.DecisionTreeClassifier(criterion='gini', random_state=42, min_samples_split=20)
clf = clf.fit(trn_xs, trn_y)

Let's draw the tree.

In [ ]:
#| code-fold: false
annotations = tree.plot_tree(clf, 
               filled=True, 
               rounded=True,
               feature_names=trn_xs.columns,
               class_names=['No', 'Yes'])

## Min Samples Split -- Evaluation Error

Let's see how it does on the validation set.

In [ ]:
#| code-fold: false
preds = clf.predict(val_xs)
mean_absolute_error(val_y, preds)

## Decision Tree -- Maximum Depth

Let's train the decision tree again but with a maximum depth of 3.

In [ ]:
#| code-fold: false
clf = tree.DecisionTreeClassifier(criterion='gini', random_state=42, max_depth=3)
clf = clf.fit(trn_xs, trn_y)

Let's draw the tree.

In [ ]:
#| code-fold: false
annotations = tree.plot_tree(clf, 
               filled=True, 
               rounded=True,
               feature_names=trn_xs.columns,
               class_names=['No', 'Yes'])

## Maximum Depth -- Evaluation Error

Let's see how it does on the validation set.

In [ ]:
#| code-fold: false
preds = clf.predict(val_xs)
mean_absolute_error(val_y, preds)

## Random Forest

Let's try a random forest.

In [ ]:
#| code-fold: false
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf = clf.fit(trn_xs, trn_y)

Let's see how it does on the validation set.

In [ ]:
#| code-fold: false
preds = clf.predict(val_xs)
mean_absolute_error(val_y, preds)

::: {.content-visible when-profile="slides"}

## Recap

* Decision Trees
* Impurity Measures
* Avoiding Overfitting
* Random Forests

:::

## References

::: {#refs}
:::
